In [68]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from holoviews import opts
import panel as pn
pn.extension()
hv.extension('bokeh')
import geopandas as gpd
from shapely.geometry import Point
import folium



In [69]:
df=pd.read_csv('data/CSVLog_20240802_AM.CSV')
df.columns = df.columns.str.strip()
df.head()

,Time (sec),Instant fuel economy (MPG),Intake manifold absolute pressure (inHg),Engine RPM (RPM),Vehicle speed (MPH),Relative throttle position (%),Ambient air temperature (°F),Fuel type,Relative accelerator pedal position (%),Mass air flow rate (lb/min),Boost (psi),Engine Power (hp),Trip Distance (miles),Trip Fuel Economy (MPG),Hard Brake Count,Latitude (deg),Longitude (deg),Bearing (deg)
0,0.000,0.0,0.000000,0.00,0.0,0.000000,0.0,0,0.0,0.000000,0.000000,3.538968,0.0,0.0,0,45.036170,-92.9851,-1.0
1,1.003,0.0,10.630836,583.50,0.0,3.137255,71.6,1,0.0,0.000000,-9.282415,4.130809,0.0,0.0,0,45.036167,-92.9851,-1.0
2,2.001,0.0,10.630836,583.50,0.0,3.137255,71.6,1,0.0,0.000000,-9.282415,4.130809,0.0,0.0,0,45.036163,-92.9851,-1.0
3,3.002,0.0,11.221438,681.25,0.0,3.529412,71.6,1,0.0,0.377742,-8.992340,4.251593,0.0,0.0,0,45.036163,-92.9851,-1.0
4,3.993,0.0,11.221438,681.25,0.0,3.529412,71.6,1,0.0,0.377742,-8.992340,4.251593,0.0,0.0,0,45.036160,-92.9851,-1.0


In [70]:
column_list=df.columns.tolist()
print(column_list)

['Time (sec)', 'Instant fuel economy (MPG)', 'Intake manifold absolute pressure (inHg)', 'Engine RPM (RPM)', 'Vehicle speed (MPH)', 'Relative throttle position (%)', 'Ambient air temperature (°F)', 'Fuel type', 'Relative accelerator pedal position (%)', 'Mass air flow rate (lb/min)', 'Boost (psi)', 'Engine Power (hp)', 'Trip Distance (miles)', 'Trip Fuel Economy (MPG)', 'Hard Brake Count', 'Latitude (deg)', 'Longitude (deg)', 'Bearing (deg)']


In [71]:
df=df[['Time (sec)', 'Vehicle speed (MPH)', 'Relative accelerator pedal position (%)','Engine RPM (RPM)', 'Boost (psi)', 'Engine Power (hp)', 'Mass air flow rate (lb/min)','Intake manifold absolute pressure (inHg)', 'Ambient air temperature (°F)','Trip Distance (miles)','Instant fuel economy (MPG)', 'Trip Fuel Economy (MPG)', 'Latitude (deg)', 'Longitude (deg)','Bearing (deg)', 'Hard Brake Count', 'Relative throttle position (%)']]
df.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Hard Brake Count,Relative throttle position (%)
0,0.000,0.0,0.0,0.00,0.000000,3.538968,0.000000,0.000000,0.0,0.0,0.0,0.0,45.036170,-92.9851,-1.0,0,0.000000
1,1.003,0.0,0.0,583.50,-9.282415,4.130809,0.000000,10.630836,71.6,0.0,0.0,0.0,45.036167,-92.9851,-1.0,0,3.137255
2,2.001,0.0,0.0,583.50,-9.282415,4.130809,0.000000,10.630836,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0,0,3.137255
3,3.002,0.0,0.0,681.25,-8.992340,4.251593,0.377742,11.221438,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0,0,3.529412
4,3.993,0.0,0.0,681.25,-8.992340,4.251593,0.377742,11.221438,71.6,0.0,0.0,0.0,45.036160,-92.9851,-1.0,0,3.529412


In [72]:
# Assuming `df` is your DataFrame
df['Boost (psi)'] = df['Boost (psi)'].bfill().infer_objects()
df.loc[df['Boost (psi)'] == 0, 'Boost (psi)'] = 0.01


In [73]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Round columns 'Time (sec)', ' Engine RPM (RPM)' and 2 other columns (Number of decimals: 0)
    df = df.round({'Time (sec)': 0, 'Engine RPM (RPM)': 0, 'Engine Power (hp)': 0, 'Time (sec)': 0})
    # Round columns ' Instant fuel economy (MPG)', ' Vehicle speed (MPH)' and 4 other columns (Number of decimals: 1)
    df = df.round({'Instant fuel economy (MPG)': 1, 'Vehicle speed (MPH)': 1,'Relative throttle position (%)': 1, 'Relative accelerator pedal position (%)': 1, 'Trip Distance (miles)': 1, 'Trip Fuel Economy (MPG)': 1})
    # Round columns ' Intake manifold absolute pressure (inHg)', ' Mass air flow rate (lb/min)', ' Bearing (deg)' (Number of decimals: 2)
    df = df.round({'Intake manifold absolute pressure (inHg)': 2, 'Mass air flow rate (lb/min)': 2,'Bearing (deg)': 2})
    # Drop columns: ' Intake manifold absolute pressure (inHg)', ' Mass air flow rate (lb/min)' and 3 other columns
    df = df.drop(columns=['Hard Brake Count','Relative throttle position (%)'])
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg)
0,0.0,0.0,0.0,0.0,0.010000,4.0,0.00,0.00,0.0,0.0,0.0,0.0,45.036170,-92.9851,-1.0
1,1.0,0.0,0.0,584.0,-9.282415,4.0,0.00,10.63,71.6,0.0,0.0,0.0,45.036167,-92.9851,-1.0
2,2.0,0.0,0.0,584.0,-9.282415,4.0,0.00,10.63,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0
3,3.0,0.0,0.0,681.0,-8.992340,4.0,0.38,11.22,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0
4,4.0,0.0,0.0,681.0,-8.992340,4.0,0.38,11.22,71.6,0.0,0.0,0.0,45.036160,-92.9851,-1.0


In [74]:
df_stationary = df_clean[df_clean['Vehicle speed (MPH)'] == 0]
df_stationary.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg)
0,0.0,0.0,0.0,0.0,0.010000,4.0,0.00,0.00,0.0,0.0,0.0,0.0,45.036170,-92.9851,-1.0
1,1.0,0.0,0.0,584.0,-9.282415,4.0,0.00,10.63,71.6,0.0,0.0,0.0,45.036167,-92.9851,-1.0
2,2.0,0.0,0.0,584.0,-9.282415,4.0,0.00,10.63,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0
3,3.0,0.0,0.0,681.0,-8.992340,4.0,0.38,11.22,71.6,0.0,0.0,0.0,45.036163,-92.9851,-1.0
4,4.0,0.0,0.0,681.0,-8.992340,4.0,0.38,11.22,71.6,0.0,0.0,0.0,45.036160,-92.9851,-1.0


In [75]:
df_clean['next_mph'] = df_clean['Vehicle speed (MPH)'].shift(-1)
mask = (df_clean['Vehicle speed (MPH)'] > 0) | ((df_clean['Vehicle speed (MPH)'] == 0) & (df_clean['next_mph'] > 0))
df_moving= df_clean[mask].drop(columns=['next_mph'])
df_moving.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg)
22,24.0,0.0,1.2,1136.0,-5.511434,12.0,0.74,18.31,71.6,0.0,0.0,0.0,45.036110,-92.98511,-1.00
23,25.0,1.9,4.7,1251.0,-5.221359,15.0,1.07,18.90,71.6,0.0,1.7,0.2,45.036106,-92.98510,-1.00
24,26.0,3.7,5.1,1395.0,-7.106849,12.0,1.09,15.06,71.6,0.0,4.2,0.4,45.036095,-92.98509,161.25
25,27.0,6.8,0.0,1338.0,-10.442718,6.0,0.79,8.27,71.6,0.0,16.6,1.1,45.036076,-92.98507,151.92
26,28.0,7.5,0.0,1082.0,-11.457981,4.0,0.48,6.20,71.6,0.0,28.4,1.5,45.036057,-92.98502,133.71


In [76]:
df_moving['Mass Air/Boost']=df_moving['Mass air flow rate (lb/min)']/df_moving['Boost (psi)']
df_moving['Mass Air/manPress']=df_moving['Mass air flow rate (lb/min)']/df_moving['Intake manifold absolute pressure (inHg)']
df_moving.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
22,24.0,0.0,1.2,1136.0,-5.511434,12.0,0.74,18.31,71.6,0.0,0.0,0.0,45.036110,-92.98511,-1.00,-0.134266,0.040415
23,25.0,1.9,4.7,1251.0,-5.221359,15.0,1.07,18.90,71.6,0.0,1.7,0.2,45.036106,-92.98510,-1.00,-0.204927,0.056614
24,26.0,3.7,5.1,1395.0,-7.106849,12.0,1.09,15.06,71.6,0.0,4.2,0.4,45.036095,-92.98509,161.25,-0.153373,0.072377
25,27.0,6.8,0.0,1338.0,-10.442718,6.0,0.79,8.27,71.6,0.0,16.6,1.1,45.036076,-92.98507,151.92,-0.075651,0.095526
26,28.0,7.5,0.0,1082.0,-11.457981,4.0,0.48,6.20,71.6,0.0,28.4,1.5,45.036057,-92.98502,133.71,-0.041892,0.077419


In [77]:
mask = (df_moving['Vehicle speed (MPH)'] <= 10)
df_less_10 = df_moving[mask]
df_less_10.describe()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,1471.098039,4.252941,1.678431,960.823529,-8.397970,7.980392,0.706078,12.432353,70.188235,24.964706,10.545098,22.268627,44.899050,-93.266832,109.103529,-0.156454,0.053326
std,765.861587,2.925088,2.896295,433.245136,2.519194,8.001225,0.566311,5.129460,0.747702,13.224475,8.005331,10.965628,0.072589,0.149307,45.230955,0.292726,0.019521
min,24.000000,0.000000,0.000000,623.000000,-11.457981,4.000000,0.350000,6.200000,69.800000,0.000000,0.000000,0.000000,44.860350,-93.351920,-1.000000,-1.374027,0.035934
25%,1735.500000,1.900000,0.000000,623.000000,-10.007604,4.000000,0.430000,9.150000,69.800000,31.300000,6.350000,27.900000,44.860477,-93.342733,90.200000,-0.076747,0.039341
50%,1893.000000,3.100000,0.000000,775.000000,-9.137378,4.000000,0.440000,10.930000,69.800000,32.000000,7.000000,27.900000,44.860477,-93.341920,90.200000,-0.047059,0.045561
75%,1905.500000,6.800000,2.550000,1211.000000,-8.774783,6.500000,0.720000,11.665000,69.800000,32.000000,16.550000,27.900000,44.864519,-93.341920,147.915000,-0.046943,0.057932
max,1918.000000,9.900000,9.000000,2110.000000,-1.595415,36.000000,2.960000,26.280000,71.600000,32.000000,29.700000,28.200000,45.036110,-92.984890,180.250000,-0.035431,0.119307


In [78]:
mask = (df_moving['Vehicle speed (MPH)'] >= 10) & (df_moving['Vehicle speed (MPH)'] <= 20)
df_10_20 = df_moving[mask]
df_10_20.describe()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,1700.062500,15.542188,3.381250,1319.578125,-7.675669,13.546875,1.178594,13.902500,69.940625,29.248438,32.743750,26.189063,44.875007,-93.317523,113.354531,0.073214,0.075401
std,489.165726,2.744160,5.305579,336.687449,4.721259,15.099595,1.122391,9.612013,0.486881,8.568121,21.068489,6.153619,0.047115,0.097646,48.708258,1.301303,0.018023
min,35.000000,10.600000,0.000000,749.000000,-11.893094,3.000000,0.340000,5.320000,69.800000,0.000000,2.900000,2.700000,44.860160,-93.351920,23.030000,-3.530116,0.042889
25%,1782.750000,13.000000,0.000000,1176.250000,-11.457981,4.000000,0.407500,6.200000,69.800000,31.300000,13.925000,27.900000,44.860324,-93.345721,88.665000,-0.128932,0.067719
50%,1855.500000,16.200000,0.000000,1239.000000,-10.080122,6.000000,0.610000,9.005000,69.800000,31.800000,31.600000,27.900000,44.860360,-93.344387,91.035000,-0.044290,0.069549
75%,1875.250000,17.550000,6.100000,1349.250000,-3.698462,17.000000,1.697500,22.002500,69.800000,31.900000,49.050000,28.000000,44.864126,-93.342769,171.652500,-0.032346,0.076941
max,1891.000000,19.900000,16.900000,2546.000000,3.770981,70.000000,5.080000,37.210000,71.600000,32.000000,74.800000,28.200000,45.035995,-92.984850,223.360000,5.630718,0.136522


In [79]:
mask = (df_moving['Vehicle speed (MPH)'] >= 20) & (df_moving['Vehicle speed (MPH)'] <= 30)
df_20_30 = df_moving[mask]
df_20_30.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
35,37.0,21.7,10.2,2022.0,5.221359,65.0,5.87,40.16,71.6,0.0,4.6,3.0,45.035840,-92.984880,181.15,1.124228,0.146165
36,38.0,24.2,0.0,1795.0,0.290075,48.0,3.51,30.12,71.6,0.0,7.0,3.3,45.035725,-92.984890,181.15,12.100299,0.116534
37,39.0,26.1,0.0,1272.0,-9.862566,6.0,0.79,9.45,71.6,0.0,63.8,4.2,45.035610,-92.984900,180.96,-0.080101,0.083598
38,40.0,26.1,0.0,1208.0,-11.167906,4.0,0.50,6.79,71.6,0.0,90.9,4.7,45.035500,-92.984920,180.00,-0.044771,0.073638
39,41.0,24.2,0.0,1196.0,-11.603019,4.0,0.41,5.91,71.6,0.1,91.6,5.7,45.035393,-92.984924,179.72,-0.035336,0.069374


In [80]:
mask = df_20_30['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

4


In [81]:
mask = (df_moving['Vehicle speed (MPH)'] >= 30) & (df_moving['Vehicle speed (MPH)'] <= 40)
df_30_40 = df_moving[mask]
df_30_40.head()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
45,47.0,30.4,19.6,3030.0,8.992340,120.0,8.12,47.84,71.6,0.1,3.5,6.1,45.035053,-92.985374,260.17,0.902991,0.169732
46,48.0,33.6,19.6,3044.0,8.702264,121.0,8.26,47.25,71.6,0.1,3.8,5.9,45.035038,-92.985590,261.40,0.949178,0.174815
47,49.0,37.3,17.6,3098.0,7.541962,107.0,7.88,44.89,71.6,0.1,4.8,5.8,45.035020,-92.985820,261.40,1.044821,0.175540
1717,1719.0,39.8,0.0,1285.0,-11.748056,4.0,0.41,5.61,69.8,31.2,137.9,28.1,44.865612,-93.350930,216.44,-0.034899,0.073084
1718,1720.0,38.5,0.0,1285.0,-12.038133,3.0,0.37,5.02,69.8,31.2,151.1,28.2,44.865498,-93.351080,227.33,-0.030736,0.073705


In [82]:
mask = df_30_40['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

15


In [83]:
mask = (df_moving['Vehicle speed (MPH)'] >= 40) & (df_moving['Vehicle speed (MPH)'] <= 50)
df_40_50 = df_moving[mask]
df_40_50.describe()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,1160.666667,44.322222,5.277778,1994.222222,-5.430857,39.000000,3.051111,18.474444,69.8,20.766667,98.333333,20.666667,44.922415,-93.229190,216.888889,0.404554,0.114619
std,832.251464,2.688763,7.922559,985.083725,9.604880,52.851206,3.999633,19.554249,0.0,15.500000,74.025013,11.150000,0.084412,0.182140,34.309350,0.687481,0.054521
min,50.000000,40.400000,0.000000,1279.000000,-11.893094,4.000000,0.390000,5.320000,69.8,0.100000,4.800000,5.800000,44.865740,-93.350800,184.220000,-0.047129,0.073308
25%,52.000000,42.300000,0.000000,1319.000000,-11.893094,4.000000,0.420000,5.320000,69.8,0.100000,6.400000,5.800000,44.866043,-93.350620,188.970000,-0.038304,0.077068
50%,1714.000000,44.100000,0.000000,1369.000000,-11.457981,5.000000,0.480000,6.200000,69.8,31.100000,129.100000,28.100000,44.866390,-93.350525,202.900000,-0.034474,0.080214
75%,1716.000000,45.400000,15.300000,3212.000000,4.786245,94.000000,7.230000,39.280000,69.8,31.100000,161.600000,28.100000,45.034935,-92.986610,261.220000,0.884253,0.182714
max,1718.000000,48.500000,16.500000,3444.000000,11.603019,135.000000,10.260000,53.150000,69.8,31.100000,168.400000,28.100000,45.034992,-92.986070,261.590000,1.521025,0.193039


In [84]:
mask = df_40_50['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

5


In [85]:
mask = (df_moving['Vehicle speed (MPH)'] >= 50) & (df_moving['Vehicle speed (MPH)'] <= 60)
df_50_60 = df_moving[mask]
df_50_60.describe()

,Time (sec),Vehicle speed (MPH),Relative accelerator pedal position (%),Engine RPM (RPM),Boost (psi),Engine Power (hp),Mass air flow rate (lb/min),Intake manifold absolute pressure (inHg),Ambient air temperature (°F),Trip Distance (miles),Instant fuel economy (MPG),Trip Fuel Economy (MPG),Latitude (deg),Longitude (deg),Bearing (deg),Mass Air/Boost,Mass Air/manPress
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.0,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,851.419753,57.412346,8.530864,1935.925926,-3.772772,31.765432,2.537037,21.848642,69.8,15.103704,57.834568,25.330864,45.021368,-93.231934,250.778519,0.392280,0.110747
std,506.821859,2.051303,7.921989,316.711729,6.384853,29.869412,1.830174,12.999494,0.0,9.419772,50.218473,5.970210,0.075454,0.118209,40.409176,3.872570,0.017927
min,53.000000,50.300000,0.000000,1498.000000,-11.893094,5.000000,0.500000,5.320000,69.8,0.200000,5.600000,5.800000,44.866764,-93.350610,178.230000,-6.963705,0.089127
25%,428.000000,55.900000,0.000000,1777.000000,-10.007604,8.000000,0.950000,9.150000,69.8,7.300000,17.400000,25.000000,45.034878,-93.298935,221.190000,-0.350181,0.102419
50%,849.000000,57.800000,7.800000,1812.000000,-3.916019,22.000000,2.380000,21.560000,69.8,15.100000,36.500000,27.900000,45.054764,-93.280020,271.380000,-0.081396,0.104696
75%,906.000000,59.000000,12.900000,1918.000000,0.870226,44.000000,3.190000,31.300000,69.8,16.000000,107.200000,28.100000,45.069366,-93.124245,275.240000,0.751241,0.109763
max,1712.000000,59.700000,29.000000,3220.000000,10.877830,144.000000,7.710000,51.680000,69.8,31.100000,153.300000,28.300000,45.070023,-92.986916,296.270000,21.994275,0.198811


In [86]:
mask = df_60_70['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

NameError: name 'df_60_70' is not defined

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 60) & (df_moving['Vehicle speed (MPH)'] <= 70)
df_60_70 = df_moving[mask]
df_60_70.describe()

In [ ]:
mask = df_60_70['Instant fuel economy (MPG)'] > 100
mpg_100 = mask.sum()
print(mpg_100)

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 60)
df_grt_60 = df_moving[mask]
df_grt_60.describe()

In [ ]:
mask = (df_moving['Vehicle speed (MPH)'] >= 70)
df_grt_70 = df_moving[mask]
df_grt_70.head()

In [ ]:
def clean_data(df):
    # Drop columns: 'Hard Brake Count', 'Time (sec)' and 14 other columns
    #df = df.drop(columns=['Instant fuel economy (MPG)', 'Intake manifold absolute pressure (inHg)', 'Engine RPM (RPM)', 'Vehicle speed (MPH)', 'Ambient air temperature (°F)', 'Relative accelerator pedal position (%)', 'Intake air temperature bank 1  sensor 1 (°F)', 'Mass air flow rate (lb/min)', 'Boost (psi)', 'Engine Power (hp)', 'Engine Torque (lb•ft)', 'Trip Distance (miles)', 'Trip Fuel Economy (MPG)', 'Bearing (deg)','Mass Air/manPress'])
    # Rename column 'Latitude (deg)' to 'latitude'
    df = df.rename(columns={'Latitude (deg)': 'latitude'})
    # Rename column 'Longitude (deg)' to 'longitude'
    df = df.rename(columns={'Longitude (deg)': 'longitude'})
    return df

df_grt_70_geo = clean_data(df_grt_70.copy())
df_grt_70_geo.head()

In [ ]:
map_center = [df_grt_70_geo['latitude'].mean(), df_grt_70_geo['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=11)

# Add markers to the map
for idx, row in df_grt_70_geo.iterrows():
    folium.PolyLine(
        locations=[[row['latitude'], row['longitude']]],
        popup=row['Time (sec)'],
        weight=5,
        opacity=1,
        color='blue'
    ).add_to(m)

# Save the map to an HTML file
m.save('interactive_map.html')

In [ ]:
hvexplorer = df.hvplot.explorer()
hvexplorer

In [ ]:
df['Engine RPM (RPM)']=df['Engine RPM (RPM)']/100
df['Engine Power (hp)']=df['Engine Power (hp)']/10
df['Ambient air temperature (°F)']=df['Ambient air temperature (°F)']/10
df['Intake air temperature bank 1  sensor 1 (°F)']=df['Intake air temperature bank 1  sensor 1 (°F)']/10
#df['Relative throttle position (%)']=df['Relative throttle position (%)']/10
#df[' Boost (psi)']=df[' Boost (psi)']/10
#df[' Mass air flow rate (lb/min)']=df[' Mass air flow rate (lb/min)']/10


#df['Boost (psi)']= df['Boost (psi)']/10


In [ ]:
start_index = int(round(2227.003))  # or use int(2227.003)
end_index = int(round(2231.996))    # or use int(2255.994)

# Select rows between start_index and end_index (inclusive)
boosted = df.iloc[start_index:end_index + 1]

print(boosted.head())

In [ ]:
#boosted=df.iloc[2227:2233]
#boosted.head

In [ ]:
boosted.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Engine RPM (RPM)','Boost (psi)','Relative throttle position (%)','Vehicle speed (MPH)' ], #'Engine Power (hp)'
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Mass Air/Boost'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y=['Mass Air/manPress'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
df.hvplot(
    kind='line',
    x='Time (sec)',
    y1='Intake air temperature bank 1  sensor 1 (°F)',
    y2='Boost (psi)',
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
moving_df.describe()